<a href="https://colab.research.google.com/github/toluwee/Machine_Learning_Projects/blob/master/Horses_vs_Humans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Horses and Humans with Convolutions

### About dataset

Horses or Humans is a dataset of 300×300 images that contains 500 rendered images of various species of horse in various poses in various locations. It also contains 527 rendered images of humans in various poses and locations. Emphasis has been taken to ensure diversity of humans, and to that end there are both men and women as well as Asian, Black, South Asian and Caucasians present in the training set. The validation set adds 6 different figures of different gender, race and pose to ensure breadth of data.

More information about the dataset available [here](http://www.laurencemoroney.com/horses-or-humans-dataset/)  

## Objective 

To create a model that correct classifies images as horses or humans

## Methodology 

### Import Dataset

Dataset is loaded from Laurence Moroney's file

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

--2020-09-15 17:06:03--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 74.125.140.128, 108.177.15.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  38.1MB/s    in 3.7s    

2020-09-15 17:06:07 (38.1 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]



In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2020-09-15 17:06:07--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.133.128, 74.125.140.128, 108.177.15.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.133.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘/tmp/validation-horse-or-human.zip’

/tmp/validation-hor 100%[===================>]  10.95M  41.6MB/s    in 0.3s    

2020-09-15 17:06:08 (41.6 MB/s) - ‘/tmp/validation-horse-or-human.zip’ saved [11480187/11480187]



### Import Libraries

In [3]:
import os
import tensorflow as tf
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

## Dataset definitions

Directories containing the necessary files are referenced as such:

In [4]:
# Directory with training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with training horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Directory with training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

In [5]:
train_horse_names = os.listdir(train_horse_dir)
train_human_names = os.listdir(train_human_dir)

validation_horse_hames = os.listdir(validation_horse_dir)
validation_human_names = os.listdir(validation_human_dir)

## Model Design

To define the model: 
*  Convolutional layers are added 
* Final result flattened to feed into the densely connected layers.
* Densely connected layers added

In [6]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

The convolution layers reduce the size of the feature maps by a bit due to padding, and each pooling layer halves the dimensions.

Since this is a binary classification, sigmoid activation and binary_crossentropy loss are used so that the network output will be a single scaler between 0 and 1.

RMSprop optimization algorithm is used  because it automates the learning-rate tuning. (Adam and Adagrad too will work fine because they also automatically adapt the learning rate during training.)

In [7]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

## Data Preprocessing

Data generators that will read pictures in the source folders, convert them to float32 tensors, and feed them (with their labels) to the network are set up. 

Images are preprocessed  by normalizing the pixel values to be in the [0, 1] range (from the [0, 255] range).


In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Train

A code to initiate callback immediately a level of accuracy is achieved is used in the model

In [9]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.90):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True
callbacks = myCallback()    

The model is trained with the generated dateset

In [10]:
history = model.fit(train_generator,
                    steps_per_epoch=8, 
                    epochs=15, 
                    verbose=1,
                    validation_data = validation_generator,
                    validation_steps=8, 
                    callbacks=[callbacks]
                    )

Epoch 1/15
8/8 [==============================] - 5s 667ms/step - loss: 3.1272 - accuracy: 0.5017 - val_loss: 0.5910 - val_accuracy: 0.5859
Epoch 2/15
8/8 [==============================] - 6s 711ms/step - loss: 0.9149 - accuracy: 0.6151 - val_loss: 0.9152 - val_accuracy: 0.5039
Epoch 3/15
8/8 [==============================] - 6s 712ms/step - loss: 0.5658 - accuracy: 0.6863 - val_loss: 0.5138 - val_accuracy: 0.7422
Epoch 4/15
8/8 [==============================] - 6s 804ms/step - loss: 0.3571 - accuracy: 0.8509 - val_loss: 0.6287 - val_accuracy: 0.7734
Epoch 5/15
8/8 [==============================] - ETA: 0s - loss: 0.1288 - accuracy: 0.9555
Reached 90% accuracy so cancelling training!
8/8 [==============================] - 6s 808ms/step - loss: 0.1288 - accuracy: 0.9555 - val_loss: 1.2649 - val_accuracy: 0.7852


## Prediction

The model is used to predict images that are uploaded 

In [21]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving download (1).jpg to download (1).jpg
Saving download (2).jpg to download (2) (1).jpg
Saving download (3).jpg to download (3) (1).jpg
Saving download (4).jpg to download (4) (1).jpg
Saving download (5).jpg to download (5) (1).jpg
Saving download (6).jpg to download (6) (1).jpg
Saving download (7).jpg to download (7) (1).jpg
Saving download (8).jpg to download (8).jpg
Saving download (9).jpg to download (9).jpg
Saving download (10).jpg to download (10) (1).jpg
[0.]
download (1).jpg is a horse
[4.388261e-27]
download (2).jpg is a horse
[1.]
download (3).jpg is a human
[0.]
download (4).jpg is a horse
[0.]
download (5).jpg is a horse
[1.]
download (6).jpg is a human
[1.]
download (7).jpg is a human
[0.]
download (8).jpg is a horse
[3.838861e-32]
download (9).jpg is a horse
[0.]
download (10).jpg is a horse


## Clean Up

The following cell is run to terminate the kernel and free memory resources

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)